In [ ]:
!unzip /content/archive.zip -d Data

Archive:  /content/archive.zip
  inflating: Data/Symptom-severity.csv  
  inflating: Data/dataset.csv        
  inflating: Data/symptom_Description.csv  
  inflating: Data/symptom_precaution.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix ,classification_report
from sklearn.ensemble import RandomForestClassifier

In [ ]:
data  = pd.read_csv("/content/Data/dataset.csv")
data_sevrity = pd.read_csv("/content/Data/Symptom-severity.csv")

In [ ]:
df1 = pd.read_csv("/content/Data/symptom_precaution.csv")
df1.head()

,Disease,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
1,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
2,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
3,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
4,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths


In [ ]:
len(df1['Disease'])

41

In [ ]:
data_sevrity.head()

,Symptom,weight
0,itching,1
1,skin_rash,3
2,nodal_skin_eruptions,4
3,continuous_sneezing,4
4,shivering,5


In [ ]:
#convert data_severity to dictionnary
data_dict = data_sevrity.set_index('Symptom').T.to_dict()

<ipython-input-7-f665341f2aac>:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data_dict = data_sevrity.set_index('Symptom').T.to_dict()


In [ ]:
def remove_space_between_word(dataset):
    for col in dataset.columns:
        for i in range(len(dataset[col])):
            if (type(dataset[col][i]) == str ):
                dataset[col][i] = dataset[col][i].strip()
                dataset[col][i] = dataset[col][i].replace(" ", "_")
    return data

In [ ]:
new_df = remove_space_between_word(data)
new_df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal_infection,itching,skin_rash,nodal_skin_eruptions,dischromic__patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal_infection,skin_rash,nodal_skin_eruptions,dischromic__patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal_infection,itching,nodal_skin_eruptions,dischromic__patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal_infection,itching,skin_rash,dischromic__patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal_infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def enc(dataset):
    for ind in data_sevrity.index:
        dataset = dataset.replace(data_sevrity["Symptom"][ind] , data_sevrity["weight"][ind])
    dataset = dataset.fillna(0) # put empty cell to 0
    dataset = dataset.replace("foul_smell_of_urine" , 5)
    dataset = dataset.replace("dischromic__patches" , 6)
    dataset = dataset.replace("spotting__urination" , 6)
    return dataset

In [ ]:
ds=enc(data)

In [ ]:
ds.dtypes

Disease        object
Symptom_1       int64
Symptom_2       int64
Symptom_3       int64
Symptom_4       int64
Symptom_5       int64
Symptom_6     float64
Symptom_7     float64
Symptom_8     float64
Symptom_9     float64
Symptom_10    float64
Symptom_11    float64
Symptom_12    float64
Symptom_13    float64
Symptom_14    float64
Symptom_15    float64
Symptom_16    float64
Symptom_17    float64
dtype: object

In [ ]:
def encode_data(dataset , data_dict_weigth):
    cols = dataset.columns
    for columnName in cols:
        for i in range(len(dataset[columnName])):
            try:
            #print(data_dict[data2[columnName][i]]["weight"])
                dataset[columnName][i] = data_dict[dataset[columnName][i]]["weight"]
            except:
                pass
    dataset = dataset.fillna(0) # put empty cell to 0
    dataset = dataset.replace("foul_smell_of_urine" , 5)
    dataset = dataset.replace("dischromic__patches" , 6)
    dataset = dataset.replace("spotting__urination" , 6)
    return dataset

In [ ]:
df = encode_data(new_df , data_dict)
df.tail()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
4915,(vertigo)_Paroymsal__Positional_Vertigo,5,3,5,6,4,4,0,0,0,0,0,0,0,0,0,0,0
4916,Acne,3,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
4917,Urinary_tract_infection,6,4,5,6,0,0,0,0,0,0,0,0,0,0,0,0,0
4918,Psoriasis,3,3,3,2,2,2,0,0,0,0,0,0,0,0,0,0,0
4919,Impetigo,3,7,4,2,3,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#check if all Symptoms are replace by their weigth
names = []
for col in df.columns:
    if(col != "Disease"):
        for i in range(len(df[col])):
            if (type(df[col][i]) == str ):
                if df[col][i] not in names :
                    names.append(df[col][i])

print(" no replace are :" , names)

 no replace are : []


In [ ]:
df_data = df.drop('Disease' , axis =1)
label = data["Disease"]

In [ ]:
label[90:100]

90    Bronchial_Asthma
91    Bronchial_Asthma
92    Bronchial_Asthma
93    Bronchial_Asthma
94    Bronchial_Asthma
95    Bronchial_Asthma
96    Bronchial_Asthma
97    Bronchial_Asthma
98    Bronchial_Asthma
99    Bronchial_Asthma
Name: Disease, dtype: object

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_data, label, shuffle=True, train_size = 0.70)
randomFC = RandomForestClassifier()
randomFC.fit(x_train, y_train)
result = randomFC.predict(x_test)
print(randomFC)
print(classification_report(y_true=y_test, y_pred=result))
print('F1-score% =', f1_score(y_test, result, average='macro')*100, '|', 'Accuracy% =', accuracy_score(y_test, result)*100)

RandomForestClassifier()
                                         precision    recall  f1-score   support

(vertigo)_Paroymsal__Positional_Vertigo       1.00      1.00      1.00        36
                                   AIDS       1.00      1.00      1.00        31
                                   Acne       1.00      1.00      1.00        26
                    Alcoholic_hepatitis       1.00      1.00      1.00        40
                                Allergy       0.86      1.00      0.92        37
                              Arthritis       1.00      1.00      1.00        38
                       Bronchial_Asthma       1.00      1.00      1.00        33
                   Cervical_spondylosis       0.90      1.00      0.95        38
                            Chicken_pox       1.00      1.00      1.00        36
                    Chronic_cholestasis       1.00      1.00      1.00        35
                            Common_Cold       1.00      1.00      1.00        31
  

In [ ]:
qw=pd.DataFrame([[1,2,3,4,5,6,1,2,3,4,5,6,1,2,3,4,5]],columns=['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5',
       'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10',
       'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15',
       'Symptom_16', 'Symptom_17'])

In [ ]:
x_train.columns

Index(['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5',
       'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10',
       'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15',
       'Symptom_16', 'Symptom_17'],
      dtype='object')

In [ ]:
output = randomFC.predict(qw)

In [ ]:
output[0]

'Hepatitis_B'

In [ ]:
import pickle

In [ ]:
filename='model.pkl'
pickle.dump(randomFC,open(filename,'wb'))

In [ ]:
clff=pickle.load(open('model.pkl','rb'))

In [ ]:
result = clff.predict(x_test)
print(classification_report(y_true=y_test, y_pred=result))
print('F1-score% =', f1_score(y_test, result, average='macro')*100, '|', 'Accuracy% =', accuracy_score(y_test, result)*100)

                                         precision    recall  f1-score   support

(vertigo)_Paroymsal__Positional_Vertigo       1.00      1.00      1.00        36
                                   AIDS       1.00      1.00      1.00        31
                                   Acne       1.00      1.00      1.00        26
                    Alcoholic_hepatitis       1.00      1.00      1.00        40
                                Allergy       0.86      1.00      0.92        37
                              Arthritis       1.00      1.00      1.00        38
                       Bronchial_Asthma       1.00      1.00      1.00        33
                   Cervical_spondylosis       0.90      1.00      0.95        38
                            Chicken_pox       1.00      1.00      1.00        36
                    Chronic_cholestasis       1.00      1.00      1.00        35
                            Common_Cold       1.00      1.00      1.00        31
                           

In [ ]:
qw1=pd.DataFrame([[1,0,0,4,5,6,1,0,0,0,0,0,0,0,0,0,0]],columns=['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5',
       'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9', 'Symptom_10',
       'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14', 'Symptom_15',
       'Symptom_16', 'Symptom_17'])

In [ ]:
out = clff.predict(qw1)

In [ ]:
out[0]

'Acne'